# DS Apps 2020 @ TAU: HW0
## Getting ready with Python

### Giora Simchoni
### 2020-02-25



### Welcome

Welcome to Homework 0 in Python!

Good news: you don’t have to do anything.

I just want you to run this Jupyter notebook at your leisure in Jupyter, using Binder cloud, Docker or your local installation, and see that everything is working. Though feel free to tweak things and show me interesting stuff if you find any.

### Jupyter notebook

This is an Jupyter notebook. When you execute code within the notebook, the results appear beneath the code.

Try executing this chunk by clicking the Run button at the top of the notebook or by placing your cursor inside it and pressing Ctrl+Enter.

In [ ]:
print('One and one is', 1 + 1)

Add a new chunk by clicking the "+" button on the toolbar or by pressing B (for "below") or A (for "above"). Delete a chunk with double D.

If you like to make a chunk a "Markdown" chunk, holding only formatted text, either change its type in the drop-down menu above to "Markdown" or if the cursor is inside the chunk, press Esc, then M.

Binder Test...